In [1]:
import os
import os
import csv
import copy
import fnmatch
import collections
import numpy as np
import collections
import pandas as pd
from subprocess import call
from itertools import repeat
from itertools import dropwhile
from IPython.display import display
from pandas import read_csv, DataFrame, Index, MultiIndex

In [9]:
#--------------------------------#
# Configure directory parameters #
#--------------------------------#

dir = '/autofs/space/lilli_001/users/DARPA-Scripts/tutorials/darpa_resting_state_EH/misc'
file = 'cp_nii.txt'

raw_data = '/autofs/space/lilli_003/users/DARPA-TRANSFER/mri/'

scan_dir = '/autofs/space/lilli_003/users/DARPA-RestingState/'
params_dir = 'RS_parameters'
scan_type = 'resting'
fsd = 'rest_001'
subdir_fsd = '001'


#-------------------#
# Initiate function #
#-------------------#

def dcm2nii(dir,file,raw_data,scan_dir,scan_type,fsd,subdir_fsd,params_dir):
    subj_list = []
    raw_dcms_list=[]
    raw_subjs = []
    test=[]
    correct_subs =[]

    with open('%s/%s' % (dir, file) ,'r') as in_txt:
        for line in in_txt:
            subj_list.append(line.strip())

    with open('%s/dcm2nii.csh' % os.path.join(scan_dir, params_dir) ,'w') as dcm2nii_convert:
        for subject in subj_list:
            raw_scan_path = os.path.join(raw_data, subject, scan_type)
            for dcm_dir in os.listdir(raw_scan_path):
                if fnmatch.fnmatch(dcm_dir, '*0*'):
                    raw_subjs.append(subject)
                    scan_subdir = os.path.join(raw_scan_path,dcm_dir)
                    dcm2nii_convert.write('set files = %s/*MR*; mri_convert ${files[2]} %s/f.nii;\n' % \
                                          (scan_subdir,os.path.join(scan_dir,subject.lower()[0:5],fsd,subdir_fsd)))
        duplicates = [item for item, count in collections.Counter(raw_subjs).items() if count > 1]
        for dup in duplicates:
            print('%s has more than one DICOM directory: \n \
            (1) Manually convert DICOM to NIFTI before proceeding to the next step \n \
            (2) Remove negligible cmd from dcm2nii.csh' % dup)
    return

#----------------#
## Call function #
#----------------#

dcm2nii(dir,file,raw_data,scan_dir,scan_type,fsd,subdir_fsd,params_dir)

HC027_01 has more than one DICOM directory: 
             (1) Manually convert DICOM to NIFTI before proceeding to the next step 
             (2) Remove negligible cmd from dcm2nii.csh
HC047_01 has more than one DICOM directory: 
             (1) Manually convert DICOM to NIFTI before proceeding to the next step 
             (2) Remove negligible cmd from dcm2nii.csh


# The following code is a **DRAFT** for fully unpacking scans and then converting dcm 2 nii
please treat it as such

In [ ]:
# %%bash
os.system('set files = /autofs/space/lilli_003/users/DARPA-TRANSFER/mri/HC013_01/resting/008/*MR*; echo "${files[2]}";')
os.system('')

# set indir = /autofs/space/lilli_001/users/DARPA-Scripts/tutorials/darpa_resting_state_EH/misc/
# set subjects = $indir/cp_nii.txt
# for subject in `cat $subjects`; do
# # destdir=(/autofs/space/lilli_003/users/DARPA-TRANSFER/mri/{$subj})
#     echo subject
# # list=$(ls /008/MR*)
# # relist=${list[1]} 
# # echo $relist > $destdi/resting.nii
# done

## Set up Environment

In [ ]:
new_ARC = ('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH')
os.system('findsession -p res > FINDSESS.txt')
os.system('cd $new_ARC | touch subjlist.txt scan.txt unpacksdcmdir_1.csh unpacksdcmdir_2.csh')

transfer_scanlist=[]
subject_list=[]

unpack_log='unpack.log'
raw_cfg= 'cfg_raw.txt'
new_cfg='cfg.txt'

## Generate Subject and Scan lists

In [ ]:
##   NOTE: THIS WILL GENERATE SUBJECT AND         ##
##   SCAN LISTS FROM BOURGET - PROJECT CODE BFR   ##
##   MODIFY subjlist.txt AND scan.txt ACCORDINGLY ##

def get_scans_subjects(path):
    with open('%s/FINDSESS.txt' % path ,'r') as in_txt, \
        open('%s/subjlist.txt' % path ,'w') as sublist, \
        open('%s/scan.txt' % path ,'w') as scanlist:
        for line in in_txt:
                if 'SUBJECT' in line:
                    sub=line.strip('SUBJECT:  ')
                    sublist.write('%s \n' % sub.strip())
                    subject_list.append(sub)
                elif 'PATH' in line:
                    scan=line.strip('PATH   :')
                    scanlist.write('%s \n' % scan.strip())
                    transfer_scanlist.append(sub)
    return

get_scans_subjects(new_ARC)

## unpackdcmsdir - 1

In [ ]:
def unpacksdcmdir_step1(path):
    with open('%s/subjlist.txt' % path ,'r') as sublist, \
        open('%s/scan.txt' % path ,'r') as scanlist, \
        open('%s/unpacksdcmdir_1.csh' % path ,'w') as unpacksdcmdir_1:
        for scan,subject in zip(scanlist,sublist):
            unpacksdcmdir_1.write('unpacksdcmdir -src %s -targ %s/%s -scanonly %s/%s/%s \n' \
            % (scan.strip(), path, subject.strip(), path, subject.strip(), new_cfg))
        print "Next: source unpacksdcmdir_1.csh"
    return

unpacksdcmdir_step1(new_ARC)

#----------------------------#
# source .csh from Py - BETA #
#----------------------------#

# from subprocess import call
# os.system('source %s/unpacksdcmdir_1.csh' % new_ARC)

#  unpacksdcmdir_1 = 'unpacksdcmdir -src %s -targ %s/%s -scanonly %s/%s/%s' % (scan, new_ARC, subject, new_ARC, subject, new_cfg)
#  call(unpacksdcmdir_1,env=os.environ,shell=True)

## Create Configuration File

In [ ]:

x2={'a':[],'b':[],'c':[],'d':[]}
def read_write_cfg(path,subject,unpack_log,raw_cfg): 
    infile=os.path.join(path,subject.strip(),unpack_log)
    outfile=os.path.join(path,subject.strip(),raw_cfg)
    with open(infile,'r') as in_txt, open(outfile,'w') as out_txt:
        for line in in_txt:
            if 'ok' in line:
                c=line.strip().split()
                n=str(c[0])
                scan=str(c[1])
                num=str(c[7])
                x2['a'].append(n)
                nifti='NIFTI'
                x2['b'].append(scan)
                x2['c'].append(nifti)
                x2['d'].append(num)

def format_cfg(path,subject,raw_cfg,new_cfg):
    with open('%s/subjlist.txt' % path ,'r') as sublist:
        for subject in sublist:
            subdir = os.path.join(path,"%s/cfg.txt" % subject.rstrip(' \n'))
            with open(subdir, 'w') as form_cfg:
                for a,b,c,d in zip(x2['a'],x2['b'],x2['c'],x2['d']):
                    form_cfg.write('\t'.join([a,b,c,d]) + '\n')
                print 'find cfg.txt formatted in subject %s directory' % subject
    return

## Read Unpack Log and Write Configuration File

In [ ]:

x2={'a':[],'b':[],'c':[],'d':[]}
def read_write_cfg(path,subject,unpack_log,raw_cfg): 
    infile=os.path.join(path,subject.strip(),unpack_log)
    outfile=os.path.join(path,subject.strip(),raw_cfg)
    with open(infile,'r') as in_txt, open(outfile,'w') as out_txt:
        for line in in_txt:
            if 'ok' in line:
                c=line.strip().split()
                n=str(c[0])
                scan=str(c[1])
                num=str(c[7])
                x2['a'].append(n)
                nifti='NIFTI'
                x2['b'].append(scan)
                x2['c'].append(nifti)
                x2['d'].append(num)

def format_cfg(path,subject,raw_cfg,new_cfg):
    with open('%s/subjlist.txt' % path ,'r') as sublist:
        for subject in sublist:
            subdir = os.path.join(path,"%s/cfg.txt" % subject.rstrip(' \n'))
            with open(subdir, 'w') as form_cfg:
                for a,b,c,d in zip(x2['a'],x2['b'],x2['c'],x2['d']):
                    form_cfg.write('\t'.join([a,b,c,d]) + '\n')
                print 'find cfg.txt formatted in subject %s directory' % subject
    return

## unpackdcmsdir - 2

In [ ]:
with open('%s/unpacksdcmdir_2.csh' % new_ARC ,'w') as unpacksdcmdir_2, \
open('%s/subjlist.txt' % new_ARC ,'r') as sublist:
    subdir = os.path.join(new_ARC,"%s/cfg.txt" % subject.rstrip(' \n'))
    for scan,subject in zip(transfer_scanlist,sublist):        
        unpacksdcmdir_2.write('unpacksdcmdir -src %s -targ %s/%s -fsfast -cfg %s \n'\
        % (scan.rstrip(' \n'), new_ARC.rstrip(' \n'), subject.rstrip(' \n'), subdir))
    print "Next: source unpacksdcmdir_2.csh"

# APPENDIX / MISC

## DICOM to NIFTI

In [ ]:
scan_list=['fieldmap_BOLD','AAScout', 'DIFFUSION_HighRes_REVISED', 'fieldmap_BOLD', \
           'fieldmap_generic_DTI', 'fMRI_ASSOCIATIVE_BOLD', 'fMRI_resting_state_1', \
           'fMRI_resting_state_2', 'fMRI_REWARD_BOLD', 'localizer', 'mprage_1mm_iso']

output=[]

def dicom2nii_get_path(path):
    for subject in subject_list:
        if subject in os.listdir(path):
            for scan in scan_list:
                if scan in os.listdir(os.path.join(path,subject)):
                    insert=os.path.join(path, subject, scan)
                    subdir=os.listdir(insert)
                    for item in subdir:
                        if item.startswith('0'):
                            insert2=os.path.join(path, subject, scan, item)
                            dicom1=os.listdir(insert2)[0]
                            insert3=os.path.join(path, subject, scan, item, dicom1)
                            save_me=output.append(insert3)  
    return

dicom2nii_get_path('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/')

## Read Configuration File

In [ ]:
def read_cfg(path,subjid,doc): 
    insert=os.path.join(path, subjid, doc)
    doc=open(insert,'r' )
    print doc.read()
    
read_cfg('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/','020043MR01','unpack.log')

## Read unpack.log and write cfg.txt file
Note: Iterates through subjects txt file in lieu of array

In [ ]:
def iter_subjlist(path,subject_list,unpack_log,raw_cfg,new_cfg):
    subjects=os.path.join(path,subject_list)
    with open(subjects,'U') as slist:
        for subject in slist: 
            subject=subject.strip().split()
            print(subject)
            read_write_cfg(path,subject,unpack_log,raw_cfg)
            format_cfg(path,subject,raw_cfg,new_cfg)
    return